In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/raw_data/cars.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4058 entries, 0 to 4057
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Model Auta        4058 non-null   object
 1   Verze modelu      4021 non-null   object
 2   Cena              4058 non-null   object
 3   Datum registrace  4005 non-null   object
 4   Najezd            4058 non-null   object
 5   Stk               4058 non-null   object
 6   Stav              4058 non-null   object
 7   Vykon             4058 non-null   object
 8   Objem motoru      4058 non-null   object
 9   Typ paliva        4058 non-null   object
 10  Typ prevodovky    4058 non-null   object
 11  Typ karoserie     4058 non-null   object
 12  Pocet dveri       4058 non-null   object
 13  Pocet mist        4058 non-null   object
 14  Barva exterieru   4058 non-null   object
 15  Adresa prodejce   4030 non-null   object
dtypes: object(16)
memory usage: 507.4+ KB
None


In [3]:
#Cisteni atributu "Model Auta" a "Verze modelu"
df["Model Auta"] = df["Model Auta"].str.replace("BMW ", "") #Odstraneni "BMW" z retezce

slouceni_modelu = { #promenna slouží ke prejmenovani nekterych kategorii
    "M2": "Řada 2",
    "M3": "Řada 3",
    "M4": "Řada 4",
    "M5": "Řada 5",
    "M6": "Řada 6",
    "M8": "Řada 8",
    "i4": "Řada 4",
    "i5": "Řada 5",
    "i7": "Řada 7",
    "iX1": "X1",
    "iX2": "X2",
    "iX3": "X3"
}

df["Model Auta"] = df["Model Auta"].replace(slouceni_modelu) #proces prejmenovani jeste v datovem typu string

df["Model Auta"] = df["Model Auta"].astype("category") #prevod na kategorialni typ

df = df.drop(columns=["Verze modelu"]) #Odstraneni sloupce "Verze modelu"

In [4]:
print(df["Model Auta"].cat.categories)

Index(['Ostatní', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'XM', 'Z3', 'Z4',
       'i3', 'i8', 'iX', 'Řada 1', 'Řada 2', 'Řada 3', 'Řada 4', 'Řada 5',
       'Řada 6', 'Řada 7', 'Řada 8'],
      dtype='object')


In [5]:
#Cisteni sloupce Cena
df["Cena"] = df["Cena"].str.replace(" ", "") #Odstranime prazdne hodnoty
df["Mena"] = df["Cena"].str.extract(r"(Kč|EUR)", expand=False) #Vytvorime atribut Mena
df["Cena_cislo"] = df["Cena"].str.extract(r"(\d+)", expand=False) #Cena bez meny
df["Cena_cislo"] = pd.to_numeric(df["Cena_cislo"], errors="coerce").astype("Int64")
df.loc[df["Mena"] == "EUR", "Cena_v_Kc"] = df["Cena_cislo"] * 25 #Vyvorime atribut Cena v Kc a prevedeme ceny v EUR do CZK
df.loc[df["Mena"] == "Kč", "Cena_v_Kc"] = df["Cena_cislo"] #Pridame ceny v korunach
df["Cena"] = df[("Cena_v_Kc")]

#Odstraneni zbytecnych sloupcu
df = df.drop(columns=["Cena_cislo"])
df = df.drop(columns=["Cena_v_Kc"])
df = df.drop(columns=["Mena"])

#Vyporadame se s prazdnymi hodnotami
cena_model_avg = df.groupby("Model Auta", observed=True)["Cena"].transform("mean").astype("Int64")
df["Cena"] = df["Cena"].fillna(cena_model_avg)

In [6]:
#Stk, pocet mist, pocet dveri, typ karoserie
print(df["Stk"].describe())

#Odstraneni vsech 3 atributu
df = df.drop(columns=["Stk"])
df = df.drop(columns=["Pocet dveri"])
df = df.drop(columns=["Pocet mist"])

#Cistení atributu "typ karoserie"
df["Typ karoserie"] = df["Typ karoserie"].astype("category")
df.loc[df["Typ karoserie"] == "VAN", "Typ karoserie"] = "MPV"
df.loc[df["Model Auta"].isin(["X1", "X2", "X3", "X4", "X5", "X6", "X7", "iX"]), "Typ karoserie"] = "SUV"
df.loc[df["Typ karoserie"] == "limuzína", "Typ karoserie"] = "sedan"
df.loc[df["Model Auta"].isin(["Řada 1", "i3"]), "Typ karoserie"] = "hatchback"
df.loc[df["Model Auta"].isin(["Řada 3", "Řada 5"]) &
       (df["Typ karoserie"].isin(["hatchback", "SUV"])), "Typ karoserie"] = "kombi"
df.loc[(df["Model Auta"] == "Řada 2") &
       (df["Typ karoserie"].isin(["hatchback", "kombi"])), "Typ karoserie"] = "MPV"
df.loc[(df["Model Auta"].isin(["Řada 2", "Řada 8", "Řada 7"])) &
       (df["Typ karoserie"].isin(["liftback", "SUV"])), "Typ karoserie"] = "sedan"
df.loc[(df["Model Auta"].isin(["Řada 4", "Řada 6"])) &
       (df["Typ karoserie"].isin([ "hatchback", "kombi"])), "Typ karoserie"] = "liftback"

#Nahrazeni kategorie "neuvedeno" modem podle modelu auta
df.loc[df["Typ karoserie"] == "neuvedeno", "Typ karoserie"] = (
    df.groupby("Model Auta", observed=True)["Typ karoserie"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

count          4058
unique           79
top       neuvedeno
freq           1982
Name: Stk, dtype: object


In [7]:
#Zobrazeni kategorii atributu "Typ karoserie"
print(df["Typ karoserie"].cat.categories)

Index(['MPV', 'SUV', 'VAN', 'hatchback', 'kabriolet', 'kombi', 'kupé',
       'liftback', 'limuzína', 'neuvedeno', 'sedan'],
      dtype='object')


In [8]:
#Cisteni atributu "Stav", "Najezd" a "Datum registrace"
df["Stav"] = df["Stav"].astype("category")
print(df["Stav"].cat.categories)

#Cisteni datumu registrace
df = df.rename(columns={"Datum registrace": "Rok"})
df["Rok"] = df["Rok"].str.extract(r"(\d{4})", expand=False) #Ponechame pouze rok
df["Rok"] = pd.to_datetime(df["Rok"], format="%Y")
df["Rok"] = df["Rok"].dt.strftime("%Y")
#Doplneni chybejicich hodnot
df.loc[(df["Stav"] == "Nové vozidlo") & (df["Rok"].isna()), "Rok"] = 2025
df.loc[(df["Model Auta"] == "i3") & (df["Rok"].isna()), "Rok"] = 2018
df.loc[(df["Model Auta"] == "X4") & (df["Rok"].isna()), "Rok"] = 2024

#Cisteni najezdu
df["Najezd"] = df["Najezd"].str.extract(r"(\d*(?=km\b))", expand=False)
df["Najezd"] = pd.to_numeric(df["Najezd"], errors="coerce").astype("Int64")
#Doplnění chybějících hodnot
df.loc[(df["Stav"] == "Nové vozidlo") & (df["Najezd"].isna()), "Najezd"] = 0
najezd_model_avg = df.groupby("Model Auta", observed=True)["Najezd"].transform("mean").astype("Int64")
df["Najezd"] = df["Najezd"].fillna(najezd_model_avg) #Doplneni prazdnych hodnot prumerem pro konkretni model

Index(['Dobrý', 'Nové vozidlo', 'Perfektní', 'Předváděcí vozidlo',
       'Uspokojivý', 'Velmi dobrý', 'Způsobilý k provozu', 'neuvedeno'],
      dtype='object')


In [9]:
# Nejdrive upravime format sloupcu "Typ paliva", "Objem motoru", "Typ prevodovky" a "Vykon"
df["Typ paliva"] = df["Typ paliva"].astype("category")

df = df.rename(columns={"Objem motoru": "Objem motoru"})
df["Objem motoru"] = df["Objem motoru"].str.extract(r"(\d*\s?\d{1,3}(?=\s?cm³\b))", expand=False).str.replace(" ", "")
df["Objem motoru"] = pd.to_numeric(df["Objem motoru"], errors="coerce").astype("Int64")

df["Typ prevodovky"] = df["Typ prevodovky"].astype("category")
print(df["Typ prevodovky"].describe())

df["Vykon"] = df["Vykon"].str.extract(r"(\d{2,3})(?=\s?PS\b)", expand=False) #Ponechame vykon v konich
df["Vykon"] = pd.to_numeric(df["Vykon"], errors="coerce").astype("Int64")

#Cisteni sloupce "Objem motoru"
df.loc[(df["Typ paliva"] == "elektro") & (~df["Objem motoru"].isin([650, 647])), "Objem motoru"] = 0
df.loc[df["Objem motoru"].isna() | df["Objem motoru"].isin([1,2, 143, 184]), "Objem motoru"] = (
    df.groupby("Model Auta", observed=True)["Objem motoru"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

df.loc[(df["Objem motoru"] == 0) & (df["Model Auta"] == "Ostatní") & (df["Vykon"].notna()), "Model Auta"] = (
    df.groupby("Vykon", observed=True)["Model Auta"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

#Cisteni sloupce "Typ paliva"
df.loc[df["Typ paliva"].isin(["hybridní - nafta", "neuvedeno"]), "Typ paliva"] = (
    df.groupby(["Model Auta", "Objem motoru"], observed=True)["Typ paliva"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)
df.loc[((df["Model Auta"] == "i8") & (df["Typ paliva"] == "benzin")), "Typ paliva"] = "hybridní - benzin"

#Cisteni atributu "Typ prevodovky"
df.loc[(df["Typ paliva"].isin(["elektro", "hybridní - benzin"])), "Typ prevodovky"] = "aut. převodovka"
df.loc[df["Typ prevodovky"] == "neuvedeno", "Typ prevodovky"] = (
    df.groupby(["Model Auta", "Rok"], observed=True)["Typ prevodovky"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)
df.loc[df["Typ prevodovky"] == "neuvedeno", "Typ prevodovky"] = (
    df.groupby(["Model Auta", "Typ paliva"], observed=True)["Typ prevodovky"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

#Cisteni atributu "Vykon"
df.loc[df["Vykon"].isna(), "Vykon"] = (
    df.groupby(["Model Auta", "Typ paliva"], observed=True)["Vykon"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

count                4058
unique                  4
top       aut. převodovka
freq                 3327
Name: Typ prevodovky, dtype: object


In [10]:
#Vytvorime sloupec "Metaliza"
df["Barva exterieru"] =df["Barva exterieru"].replace("neuvedeno", np.nan) #Ponechame chybejici hodnoty
df["Metaliza"] = df["Barva exterieru"].str.contains(r"\b(?:\w+\s)*metalíza\b", regex = True)

#Odstraneni "metalizy" ze sloupce "Barva exterieru" a kategorizace
df["Barva exterieru"] = df["Barva exterieru"].str.replace(" metalíza", "")
#Odstraneni retezcu oznacujicich svetle a tmave barvy
df["Barva exterieru"] = df["Barva exterieru"].str.replace("světle ", "")
df["Barva exterieru"] = df["Barva exterieru"].str.replace("tmavě ", "")
df["Barva exterieru"] = df["Barva exterieru"].astype("category")

In [11]:
#Extrahovani PSC do noveho sloupce
df["PSC"] = df["Adresa prodejce"].str.extract(r"(\d{5})", expand=False)
df["PSC"] = pd.to_numeric(df["PSC"], errors="coerce").astype("Int64")
df.loc[df["Adresa prodejce"].notna() & df["PSC"].isna() &
       df['Adresa prodejce'].str.contains('Praha', na=False), "PSC"] = 16000

#Nový DataFrame pro spojeni pridani slopce kraje podle PSC
df_kraje = pd.read_csv("../data/raw_data/zv_cobce_psc.csv", encoding='windows-1250', sep=';', usecols=["psc", "nazevkraj"])

df_kraje = df_kraje.rename(columns={"psc": "PSC"})
df_kraje = df_kraje.drop_duplicates(subset="PSC")
df = pd.merge(df, df_kraje, on="PSC", how="left")
df = df.rename(columns={"nazevkraj": "Kraj"})

#Nahrazeni prazdnych hodnot
df.loc[(df["PSC"] > 80000) | (df["PSC"] < 10000), "Kraj"] = "Slovensko"
df.loc[(df['PSC'] == 10205), "Kraj"] = "Hlavní město Praha"
df.loc[(df['PSC'] == 77200), "Kraj"] = "Olomoucký"
df.loc[(df['PSC'] == 46675), "Kraj"] = "Jihočeský"

#Prevod kraje na kategorialni datovy typ
df["Kraj"] = df["Kraj"].astype("category")

#Odstraneni pomocnych sloupcu
df = df.drop(columns=["Adresa prodejce"])
df = df.drop(columns=["PSC"])

#Vypocet prumerne ceny vozidel ze Slovenska a z Česka
print(df[df["Kraj"] == "Slovensko"]["Cena"].mean()) #Slovensko
print(df[~df["Kraj"].isin(["Slovensko"])]["Cena"].mean()) #Česko
df = df[~df["Kraj"].isin(["Slovensko"])] #Odstaneni zaznamu ze Slovenska

1633644.0404040404
1218121.180601162


In [12]:
df = df[~df["Model Auta"].isin(["Ostatní"])]
df = df[~df["Typ prevodovky"].isin(["poloaut. převodovka"])]
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3954 entries, 0 to 4057
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   model_auta       3954 non-null   category
 1   cena             3954 non-null   Int64   
 2   rok              3954 non-null   object  
 3   najezd           3954 non-null   Int64   
 4   stav             3954 non-null   category
 5   vykon            3954 non-null   Int64   
 6   objem_motoru     3954 non-null   Int64   
 7   typ_paliva       3954 non-null   category
 8   typ_prevodovky   3954 non-null   category
 9   typ_karoserie    3954 non-null   category
 10  barva_exterieru  3890 non-null   category
 11  metaliza         3890 non-null   object  
 12  kraj             3924 non-null   category
dtypes: Int64(4), category(7), object(2)
memory usage: 261.9+ KB
None


In [13]:
df.to_csv("../data/processed_data/1_data_clearing/1_clearing_cars.csv", index=False, encoding="utf-8")